<h1><a id='outline'>Project Outline</a></h1>

* [Sequencing Summaries](#seqSums)
    *  [Adapter Trimming and QC (Trimmomatic) Report](#readbd)
    *  [Host Plant Alignment Report](#pAlign)
    *  [Switchgrass Fungal Alignments](#swAlign)
    *  [Alignment Conclusions](#aConc)
* [Annotation Report](#anno)   


* [NMDS of annotated contigs](#nmds)
  *  [All Crops](#anmds)
  *  [Switchgrass](#sgnmds)
  *  [Miscanthus](#msnmds)
*  [Function Analysis](#metaT)

In [1]:
from ipywidgets import Text, Dropdown, Output, AppLayout, interact  # If not already imported

output_radio_selected = Text() # Used to take the user input and access it when needed
displayType = Dropdown(
    options=["Percentage","Count"],
    value='Percentage',
    description='Reads by:',
    disabled=False,
)
# radio_input = widgets.RadioButtons(options=['Option 1', 'Option 2']) # Declare the set of radio buttons and provide options
output = Output()

@output.capture()
def on_change_displayType(sender): # Connect the input from the user to the output so we can access it
    output.clear_output()
#     print(sender)
    global selected_option # Global variable to hold the user input for reuse in your code
    output_radio_selected.value = displayType.value
    selected_option = output_radio_selected.value # Example variable assigned the selected value
    print('Selected option set to: ' + selected_option) # For test purposes

displayType.observe(on_change_displayType, names=['value']) # Run the bind... function when the radio button is changed
displayType # Display the radio buttons to the user
display(displayType)
display(output)

Dropdown(description='Reads by:', options=('Percentage', 'Count'), value='Percentage')

Output()

In [5]:

def square(x):
    return x * x
interact(square, x=(0, 100, 10));

interactive(children=(IntSlider(value=50, description='x', step=10), Output()), _dom_classes=('widget-interact…

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# @hidden_cell
# %matplotlib inline
from ipywidgets import AppLayout, Button, Output, Dropdown
from IPython.display import display
# top = Button(description="Top")

top = Dropdown(
    options=["Percentage","Count"],
    value='Percentage',
    description='Reads by:',
    disabled=False,
)
# bottom = Button(description="Bottom")
# l = AppLayout(header=top, footer=bottom )
display(top)

output = Output()

@output.capture()
def helloWorld(button):
    print(dir(button))
    output.clear_output();
    print("Hello World")
    if button.new == "Count":
        print("Count")
    else:
        print("Percentage")
#     print(button.variables[button][:])
#     print(dir(button))
#     print(button)
#     print(button.new)

top.observe(helloWorld)

display(output)


Dropdown(description='Reads by:', options=('Percentage', 'Count'), value='Percentage')

Output()

In [ ]:
from ipywidgets import interact
languages = {"perc":"tage",'count':'the reads'}
def my_function(x):
    return x
l = interact(my_function, x=languages,description="Language")
display(l)

In [ ]:
languages = [('Python', 'Rocks!'), ('C++', 'is hard!')]
interact(my_function, x=languages)

In [ ]:
top.on_trait_change

In [ ]:
from IPython.display import display
from ipywidgets import widgets
button = widgets.Button(description="Click Me!")
display(button)

output = widgets.Output()

@output.capture()
def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)
display(output)

In [ ]:
l

In [ ]:

from IPython.display import display
display(l)

<h2><a id="seqSums">Sequencing Summaries</a></h2>
<h4><a id="readbd">Adapter Trimming and QC (Trimmomatic) Report</a></h4>

![alt text](images/TrimReport.png "Title")

<h4><a id="pAlign">Alignments</a></h4>

![alt text](images/PlantAlignedReads.png "Plant Aligned Reads")


<h4><a id="swAlign">Switchgrass Fungal alignment</a></h4>

![alt text](images/SwitchgrassFungalAlign.png "Title")


<h4><a id="aConc">Annotation Report</a></h4>
<p>Annotations were preformed using PROKKA annotations with gene identifiers mapped to KEGG Pathways. Exact annotation steps explained <a href='scripts/hpc_scripts/AnnotationPreprossesing.sh'>here</a>. The mapping and extraction of annotation counts can be found <a href='scripts/hpc_scripts/NGS_Mapping.sh'>here</a>.</p>

<h4><a id="aConc">Alignment &amp; Annotation Observations</a></h4>
<ol>
    <li>The percentage of plant reads sequenced is high thoughout the season and tapers off towards the end of the season. This may be do to senescence of plant cells.</li>
    <li>Alignment to the 6 selected fungal assemblies is low, but picks up during the warmer months. Overall, reads aligning to the combined fungal assembly were &lt;10% of the non-plant reads.</li>
    <li>The overlap between which contigs have both a prokaryotic and a eukaryotic annotation suggest either shared gene similarity or eukarotic contamination.</li>
</ol>

<h4>Is there a seasonal pattern of functional selection?</h4>

In [ ]:
%%bash
ls images